## 필요한 패키지

In [2]:
import numpy as np
import pandas as pd
import re
import csv

## AI Hub 감성대화 말뭉치 데이터

출처 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=86

불안    10433
분노    10417
상처    10150
슬픔    10128
당황     9804
기쁨     7339

## AI Hub 한국어 단발성,연속적 대화 데이터셋
출처 : https://aihub.or.kr/aihubdata/data/view.do?dataSetSn=271

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830

출처 : https://aihub.or.kr/aihubdata/data/view.do?dataSetSn=270

In [3]:
# 엑셀 파일 읽기
df1 = pd.read_excel('/home/kdt-admin/data/감성대화말뭉치(최종데이터)_Training.xlsx',index_col=0)  # traing_data 51630
df2 = pd.read_excel('/home/kdt-admin/data/감성대화말뭉치(최종데이터)_Validation.xlsx',index_col=0)  # validation_data 6641

In [4]:
# 짧은 대화도 있으므로 NAN값을 모두 공백으로 변경
df1 = df1.fillna("")
df2 = df2.fillna("")

df1["사람문장"]=df1["사람문장1"].astype(str)+df1["사람문장2"].astype(str)+df1["사람문장3"].astype(str)
df2["사람문장"]=df2["사람문장1"].astype(str)+df2["사람문장2"].astype(str)+df2["사람문장3"].astype(str)

In [5]:
df = pd.concat([df1, df2], ignore_index=True)
df = df[["감정_대분류","사람문장"]]
df.rename(columns={'사람문장':'문장','감정_대분류':"감정"},inplace=True)

#컬럼 내 데이터들의 공백(string) 제거
df["감정"] = df["감정"].apply(lambda x:x.strip())

df["감정"] = np.where(df["감정"].str.match("상처"),"슬픔", df["감정"])
df["감정"] = np.where(df["감정"].str.match("기쁨"),"행복", df["감정"])

In [6]:
df["감정"].value_counts()

감정
슬픔    20278
불안    10433
분노    10417
당황     9804
행복     7339
Name: count, dtype: int64

In [7]:
df

,감정,문장
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...
...,...,...
58266,당황,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...
58267,당황,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아...
58268,당황,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.요...
58269,당황,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...


In [8]:
df3 = pd.read_excel('/home/kdt-admin/data/한국어_단발성_대화_데이터셋.xlsx')
df4 = pd.read_excel('/home/kdt-admin/data/한국어_연속적_대화_데이터셋.xlsx')

In [9]:
df3 = df3.iloc[:,:2]
df3.rename(columns={'Sentence':"문장",'Emotion':'감정'},inplace=True)

In [10]:
df3 = df3[df3["감정"]!="놀람"]
df3["감정"] = np.where(df3["감정"].str.match("공포"),"불안", df3["감정"])
df3["감정"].value_counts()

감정
행복    6037
분노    5665
불안    5468
혐오    5429
슬픔    5267
중립    4830
Name: count, dtype: int64

In [11]:
# 첫행을 컬럼으로 바꾸기
new_columns = df4.iloc[0]
df4 = df4.rename(columns=new_columns).iloc[:,1:3]

In [12]:
df4.drop(index=0,axis=0,inplace=True)

In [13]:
df4["감정"].value_counts()

감정
중립     43786
놀람      4866
분노      3628
슬픔      1972
행복      1030
혐오       220
공포        98
ㅍ         12
분          4
ㅈ중립        1
분ㄴ         1
중림         1
ㄴ중립        1
줄          1
Name: count, dtype: int64

In [14]:
df4 = df4[df4["감정"].isin(["중립","놀람","분노","슬픔","행복","혐오","공포"])]

In [15]:
# 감정 Label 일정하게 맞춰주기 (감성대화 말뭉치와 통합을 위해)

df4["감정"] = np.where(df4["감정"].str.match("놀람"),"당황", df4["감정"])
df4["감정"] = np.where(df4["감정"].str.match("공포"),"불안", df4["감정"])
df4.rename(columns={'발화':'문장'},inplace=True)
df4["감정"].value_counts()

감정
중립    43786
당황     4866
분노     3628
슬픔     1972
행복     1030
혐오      220
불안       98
Name: count, dtype: int64

In [19]:
df_add = pd.read_csv('/home/kdt-admin/data/sentiment_augmentaition.csv',index_col=0)

In [21]:
final_df = pd.concat([df,df3,df4,df_add])

In [22]:
final_df.reset_index(drop=True,inplace=True)

In [24]:
final_df["감정"] = np.where(final_df["감정"].str.match("혐오"),"분노", final_df["감정"])

In [27]:
final_df["감정"].value_counts()

감정
중립    48616
행복    28313
슬픔    27517
불안    27398
분노    25359
당황    21273
Name: count, dtype: int64

In [19]:
final_df.to_csv("final_sentiment_dialogues.csv")